In [29]:
import gym
import random
import tflearn

import numpy as np
from tflearn.layers.core import input_data, dropout , fully_connected
from tflearn.layers.estimator import regression
from statistics import mean,median

In [30]:
LR = 1e-3 #learning rate
env = gym.make('CartPole-v0')
env.reset()
goal_steps = 200
score_requirement = 50
initial_games = 2000


[2017-09-13 15:39:57,804] Making new env: CartPole-v0


In [31]:
def some_random_game_first():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
            env.render()
            action = env.action_space.sample()
            observation,reward,done,info = env.step(action)
            if done:
                break
#some_random_game_first()

In [32]:
def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        
        for _ in range(goal_steps):
            action = random.randrange(0,2)
            observation,reward,done,info = env.step(action)
            
            if len(prev_observation) > 0:
                game_memory.append([prev_observation,action])
            
            prev_observation = observation
            score += reward
            if done: 
                break
        if score > score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                training_data.append([data[0],output])
        env.reset()
        scores.append(score)
    
    training_data_save = np.array(training_data)
    np.save('saved.npy',training_data_save)
    
    print(accepted_scores)
    print("avg score: " , mean(accepted_scores))
    return training_data

#initial_population()

In [33]:
def neural_network_model(input_size):
    network = input_data(shape = [None,input_size,1],name = "input")
    
    network = fully_connected(network,128,activation = "relu")
    network = dropout(network,0.8)
    
    network = fully_connected(network,128,activation = "relu")
    network = dropout(network,0.8)
    
    network = fully_connected(network,128,activation = "relu")
    network = dropout(network,0.8)
    
    network = fully_connected(network,128,activation = "relu")
    network = dropout(network,0.8)
    
    network = fully_connected(network,128,activation = "relu")
    network = dropout(network,0.8)
    
    network = fully_connected(network,2,activation = "softmax")
    network - regression(network,optimizer = "adam",learning_rate = LR, loss= "categorical_crossentropy", name = "targets")

    
    model = tflearn.DNN(network,tensorboard_dir="log")
    
    return model

In [34]:
def train_model(training_data, model= False):
    x = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
    y = [i[1] for i in training_data]
    
    
    if not model:
        model = neural_network_model(input_size = len(x[0]))
        
    model.fit({'input': x}, {'targets': y}, n_epoch=5, snapshot_step=500, show_metric=True, run_id='openai_learning1')
    return model



In [35]:
training_data = initial_population()
model = train_model(training_data)

[74.0, 53.0, 61.0, 53.0, 56.0, 94.0, 51.0, 53.0, 57.0, 78.0, 58.0, 55.0, 51.0, 61.0, 53.0, 53.0, 68.0, 55.0, 97.0, 62.0, 56.0, 59.0, 51.0, 63.0, 59.0, 56.0, 51.0, 51.0, 94.0, 57.0, 73.0, 59.0, 65.0, 51.0, 55.0, 53.0, 62.0, 60.0, 61.0, 53.0, 62.0, 67.0, 65.0, 56.0, 54.0, 59.0, 70.0, 60.0, 57.0, 53.0, 57.0, 57.0, 51.0, 52.0, 58.0, 52.0]
('avg score: ', 60.035714285714285)
---------------------------------
Run id: openai_learning
Log directory: log/
INFO:tensorflow:Summary name Accuracy/Adam_0_0 (raw) is illegal; using Accuracy/Adam_0_0__raw_ instead.


[2017-09-13 15:40:58,944] Summary name Accuracy/Adam_0_0 (raw) is illegal; using Accuracy/Adam_0_0__raw_ instead.


INFO:tensorflow:Summary name Accuracy_1/Adam_1_0 (raw) is illegal; using Accuracy_1/Adam_1_0__raw_ instead.


[2017-09-13 15:40:58,961] Summary name Accuracy_1/Adam_1_0 (raw) is illegal; using Accuracy_1/Adam_1_0__raw_ instead.


INFO:tensorflow:Summary name Accuracy_2/Adam_0_1 (raw) is illegal; using Accuracy_2/Adam_0_1__raw_ instead.


...

[2017-09-13 15:40:59,005] Summary name Accuracy_3/Adam_1_1 (raw) is illegal; using Accuracy_3/Adam_1_1__raw_ instead.


INFO:tensorflow:Summary name Accuracy_4/Adam (raw) is illegal; using Accuracy_4/Adam__raw_ instead.


[2017-09-13 15:40:59,029] Summary name Accuracy_4/Adam (raw) is illegal; using Accuracy_4/Adam__raw_ instead.


---------------------------------
Training samples: 16530
Validation samples: 0
--


InvalidArgumentError: You must feed a value for placeholder tensor 'input_1/X' with dtype float
	 [[Node: input_1/X = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'input_1/X', defined at:
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/amikar/Library/Python/2.7/lib/python/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instan

In [37]:
scores = []
choices = []
for each_game in range(10):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        env.render()

        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1,len(prev_obs),1))[0])

        choices.append(action)
                
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score+=reward
        if done: break

    scores.append(score)
print('Average Score:',sum(scores) , len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)len(choices)))
print(score_requirement)

('Average Score:', 1930.0, 10)
choice 1:958  choice 0:1930
50
